# 株価データの取得と保存
このノートブックでは、companiesテーブルに登録された企業のYahoo Financeティッカーを用いて株価データを取得し、PostgreSQLのstock_pricesテーブルに保存します。

## ライブラリのインポート

In [ ]:
import os
from datetime import datetime
import pandas as pd
import yfinance as yf
from sqlalchemy import create_engine, text
from sqlalchemy.engine import URL


## データベースからティッカー一覧を取得

In [ ]:
# .envから接続情報を読み込み
from dotenv import load_dotenv
load_dotenv('../.env')

# SQLAlchemy用の接続URLを作成
database_url = URL.create(
    drivername='postgresql+psycopg2',
    username=os.getenv('POSTGRES_USER'),
    password=os.getenv('POSTGRES_PASSWORD'),
    host=os.getenv('POSTGRES_HOST', 'localhost'),
    port=os.getenv('POSTGRES_PORT', '5432'),
    database=os.getenv('POSTGRES_DB'),
)
engine = create_engine(database_url)

# companiesテーブルから company_id と yahoo_ticker を取得
df_companies = pd.read_sql('SELECT id as company_id, yahoo_ticker FROM companies WHERE yahoo_ticker IS NOT NULL', engine)
tickers = df_companies['yahoo_ticker'].tolist()
print(f'{len(tickers)} tickers loaded')


## yfinanceで株価データを取得

In [ ]:
start = '2015-01-01'
end = datetime.today().strftime('%Y-%m-%d')

# 一度に大量ティッカーを渡すと失敗することがあるため、50件ずつに分割
chunks = [tickers[i:i+50] for i in range(0, len(tickers), 50)]
all_data = []
for chunk in chunks:
    try:
        data = yf.download(' '.join(chunk), start=start, end=end, threads=True, group_by='ticker', auto_adjust=False)
    except Exception as e:
        print(f'failed to download chunk {chunk}: {e}')
        continue
    if isinstance(data.columns, pd.MultiIndex):
        for ticker in chunk:
            if ticker in data.columns.levels[1]:
                df = data.xs(ticker, level=1, axis=1).copy()
                df['ticker'] = ticker
                all_data.append(df.reset_index())
    else:
        data['ticker'] = chunk[0]
        all_data.append(data.reset_index())

stock_df = pd.concat(all_data, ignore_index=True)
print(stock_df.head())


## データ整形

In [ ]:
# companies情報と結合してcompany_idを付与
stock_df = stock_df.merge(df_companies, left_on='ticker', right_on='yahoo_ticker', how='left')
stock_df.rename(columns={
    'Date': 'date',
    'Open': 'open',
    'High': 'high',
    'Low': 'low',
    'Close': 'close',
    'Adj Close': 'adj_close',
    'Volume': 'volume'
}, inplace=True)
stock_df = stock_df[['company_id', 'date', 'open', 'high', 'low', 'close', 'adj_close', 'volume']]
stock_df.dropna(subset=['company_id'], inplace=True)


## データベースへ保存

In [ ]:
# stock_pricesテーブルにデータを書き込む
# テーブルが存在しなければ作成、既存なら追記モード
stock_df.to_sql('stock_prices', con=engine, if_exists='append', index=False, method='multi', chunksize=1000)
print('✅ データ保存完了')
